In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import requests
import json
import requests
from tqdm import tqdm

In [2]:
# ---------------------- INICIALIZACIÓN ----------------------

# Intenta leer el archivo CSV para obtener las URLs ya procesadas.
# Si no existe el archivo, crea uno nuevo.
try:
    actores = pd.read_csv("Datos/data_actores.csv")
except FileNotFoundError:
    # Crear el archivo CSV con encabezados si no existe
    with open("Datos/data_actores.csv", "w") as file:
        file.write("Nombre,Descripción,Id_IMDB")

ruta_actores = 'Datos/data_actores.csv'
actores = pd.read_csv(ruta_actores)

ruta_archivo = 'Datos/data_peliculas_corregido.csv'
datos = pd.read_csv(ruta_archivo)

FileNotFoundError: [Errno 2] No such file or directory: 'Datos/data_actores.csv'

In [ ]:
urls_procesadas = []
# Crear el archivo CSV con encabezados si no existe
if 'Procesado_IMDB' not in datos.columns:
    datos['Procesado_IMDB'] = None
datos['Procesado_IMDB'] = 'No'

In [ ]:
url = "https://imdb-api.com/API/AdvancedSearch/k_hy41rb6h"

In [20]:
actores

,Nombre,Descripcion,Id_IMDB
0,Mary Carrillo,"Actress, The Little Apartment (1958)",nm0140518
1,Alfredo Landa,"Actor, El crack (1981)",nm0484376
2,Francisco Rabal,"Actor, Tie Me Up! Tie Me Down! (1989)",nm0704719
3,Juan Diego Botto,"Actor, La Celestina (1996)",nm0098709
4,Terele Pávez,"Actress, La Celestina (1996)",nm0701715
...,...,...,...
2204,Julio Manrique,"Actor, Parallel Mothers (2021)",nm0991890
2205,Lluís Villanueva,"Actor, Porca misèria (2004–2007)",nm1258766
2206,Joaquín Notario,"Actor, Julieta (2016)",nm0636541
2207,Éric Caravaca,"aka ""Eric Caravaca""",nm0136001


In [ ]:
# Contador de actores procesados
actores_procesados = 0

actores_peli = actores.copy().head()
for indice, fila in tqdm(datos.iterrows(), total=datos.shape[0]):
    reparto = fila['Reparto']

    # Si ya ha sido procesado, saltar a la siguiente iteración
    if fila['Procesado_IMDB'] == 'Sí':
        continue
    try:
        nombres = reparto.split(", ")
        
        for nombre in nombres:
            if nombre not in actores['Nombre']:
                try:
                    # Buscar la película en TMDb
                    url_busqueda = f'https://imdb-api.com/API/SearchName/k_hy41rb6h/{nombre}'
                    #url_busqueda = f"{url}?title={titulo}&tytle_type=feature&release_date={año}-01-01,{año}-12-31"
                    #https://imdb-api.com/en/API/SearchName/k_hy41rb6h/Jean%20Reno
                    #print(url_busqueda)
                    respuesta_busqueda = requests.get(url_busqueda)
                    #print(respuesta_busqueda)
                    datos_busqueda = pd.json_normalize(respuesta_busqueda.json()["results"])
                    datos_actor = datos_busqueda.head(1)
                    #print(datos_actor)
                    if len(datos_actor) > 0:
                        name = datos_actor.get('title', 0)
                        Descripcion = datos_actor.get('description', 0)
                        imdb_id = datos_actor.get('id', 0).values[0]
                        if (imdb_id not in actores["Id_IMDB"].values) & (imdb_id not in actores_peli["Id_IMDB"].values):               
                            #print('hola')                        
                            actor= pd.DataFrame({'Nombre' : name, 'Descripcion' : Descripcion, 'Id_IMDB' : imdb_id})
                            #df = pd.DataFrame({'Title': titles, 'URL': urls, 'Procesado_Filmaffinity': 'No'})
                            #actores.at[indice, 'Descripción'] = Description
                            #actores.at[indice, 'Id_IMDB'] = imdb_id
                            actores_procesados += 1
                            actores_peli = pd.concat([actores_peli, actor], ignore_index = False).drop_duplicates("Id_IMDB")                       
                            # actores.to_csv(ruta_actores, index=False)
                            if actores_procesados % 10 == 0: 
                                print(f"Se han capturado {actores_procesados} actores de la película")
                           #else:
                            # Si no se encontró la película, marcar como no procesada
                            #datos.at[indice, 'Procesado'] = 'No'
                        #print(actores_procesados)
                    datos.at[indice, 'Procesado_IMDB'] = 'Sí'
                    datos.to_csv(ruta_archivo, index=False)
                except Exception as e:
                    print(f"Error procesando el actor123 {nombre}: {e}")
    except Exception as e:
        print(f"Error procesando el actor {nombre}: {e}")
        # Marcar la película como no procesada en el DataFrame original
        #datos.at[indice, 'Procesado_IMDB'] = 'No'
actores = pd.concat([actores, actores_peli], ignore_index = True).drop_duplicates("Id_IMDB")
# Guardar el DataFrame en el archivo CSV al final del proceso
datos.to_csv(ruta_archivo, index=False)
actores.to_csv(ruta_actores, index=False)
print(f"Datos de actores guardados en {ruta_actores}")

# Imprimir el DataFrame
print(actores)

In [28]:
###### actores = pd.concat([actores, actores_peli], ignore_index = True).drop_duplicates("Id_IMDB")
# Guardar el DataFrame en el archivo CSV al final del proceso
datos.to_csv(ruta_archivo, index=False)
actores.to_csv(ruta_actores, index=False)
print(f"Datos de actores guardados en {ruta_actores}")

Datos de actores guardados en Datos/data_actores.csv


In [ ]:
# Realiza un reset de la columna Procesado, para volver a procesar
# Leer el archivo CSV
file_path = 'data/data_peliculas_corregido.csv'
data = pd.read_csv("data/data_peliculas_corregido.csv", low_memory=False)

# Cambiar todos los valores en la columna 'Procesado' a 'No'
data['Procesado_IMDB'] = 'No'

# Guardar el archivo CSV modificado
data.to_csv(file_path, index=False)

In [ ]:
respuesta_busqueda

In [ ]:
datos_busqueda.head(1)

In [ ]:
       if datos_busqueda['results']:
            # Obtener el ID de la película
            id_pelicula = datos_busqueda['results'][0]['id']
            # Consultar los detalles de la película
            url_detalles = f'{url_base}/movie/{id_pelicula}?api_key={clave_api}'
            respuesta_detalles = requests.get(url_detalles)
            datos_detalles = respuesta_detalles.json()
            # Obtener el presupuesto y los ingresos
            presupuesto = datos_detalles.get('budget', 0)
            ingresos = datos_detalles.get('revenue', 0)
            # Incrementar el contador de películas procesadas
            peliculas_procesadas += 1
            # Actualizar el DataFrame original con los datos extraídos
            datos.at[indice, 'Presupuesto'] = presupuesto
            datos.at[indice, 'Ingresos'] = ingresos
            datos.at[indice, 'Procesado'] = 'Sí'
            # Guardar el DataFrame en el archivo CSV cada 100 películas procesadas
            if peliculas_procesadas % 100 == 0:
                datos.to_csv(ruta_archivo, index=False)
                print(f"Datos económicos de las primeras {peliculas_procesadas} películas guardados en {ruta_archivo}")
        else:
            # Si no se encontró la película, marcar como no procesada
            datos.at[indice, 'Procesado'] = 'No'

In [ ]:
endpoint = f"https://imdb-api.com/en/API/SearchName/k_hy41rb6h/María Teresa Rabal Balaguer"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)



In [ ]:
resultado = pd.json_normalize(response.json()["results"])

In [ ]:
peli = resultado.head(1)

In [ ]:
peli

In [ ]:
generos = pd.json_normalize(peli["genreList"])

In [ ]:
generos = generos.T

In [ ]:
generos

In [ ]:
dato

In [ ]:
generos_2 = generos_2.index(generos.values(

In [ ]:
reparto